In [35]:
import pandas as pd
import pyarrow.parquet as pq
import glob
from nltk import edit_distance
from tqdm import tqdm
import re
tqdm.pandas()

In [36]:
def dataframe(path, cols):
	paths = glob.glob(path)
	dataset = pq.ParquetDataset(paths)
	table = dataset.read()
	df = table.to_pandas()
	df = df[cols]
	return df

In [37]:
df = dataframe('base/*/*.parquet', ['word', 'definition', 'score'])

In [38]:
df['spaces'] = df.progress_apply(lambda row: ' ' in row.word, axis=1)

100%|██████████| 1284585/1284585 [00:31<00:00, 40399.53it/s]


In [39]:
df = df[df['spaces'] == False]

In [40]:
def strip_special(word):
    return re.sub('[^A-Za-z0-9]+', '', word).lower()

def similarity(row):
    content = [strip_special(word) for word in row["definition"].split()]
    return min([edit_distance(word, strip_special(row["word"])) for word in content])

df['similarity'] = df.progress_apply(lambda row: similarity(row), axis=1)

100%|██████████| 698110/698110 [21:43<00:00, 535.37it/s]


In [41]:
df = df.drop(df[df['similarity'] < 3].index)

In [42]:
df['pat'] = df['score'].apply(lambda s: round(s, 1))

In [43]:
df = df.drop(['spaces', 'similarity'], axis=1)

In [44]:
df.to_parquet('output', engine='pyarrow', partition_cols = ['pat'])